# MilvusDB 사용하기
milvusdb에 콜렉션을 생성하고 스키마를 정의한 후 데이터를 입력합니다.
입력한 데이터를 검색합니다.
작업이 완료되면 생성한 컬렉션을 삭제합니다.

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

!pip install -U langchain-huggingface==0.1.2
!pip install chromadb==0.4.2
!pip install pydantic==1.10.21
!pip install langchain==0.3.0
!pip install langchain --upgrade
!pip install flask-sqlalchemy --user
!pip install --upgrade pypdf
!pip install --upgrade cryptography
!pip install sentence-transformers
!pip install langchain_openai
!pip install langchain_community

!pip install ibm-cos-sdk==2.12.0
!pip install ibm-cos-sdk-core==2.12.0
!pip install ibm-cos-sdk-s3transfer==2.12.0
!pip install ibm-watson-machine-learning==1.0.356
!pip install pymilvus
!pip install -U langchain-ibm

라이브러리 설치가 완료 되었으면 노트북 커널을 Restart 합니다.
메뉴에서 `kernel > restart` 를 실행합니다.

In [ ]:
# pip show pymilvus --files

### Milvusdb  접속
Milvus 클라이언트 설치

In [ ]:
from sentence_transformers import SentenceTransformer
from pymilvus import connections,utility,Collection,CollectionSchema, FieldSchema,DataType
from langchain.vectorstores import Milvus
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.docstore.document import Document


milvus 접속 정보를 변경합니다.   

In [ ]:
MILVUS_DB_HOST = "<MILVUS_DB_HOST>" # milvusdb 호스트 정보를 변경합니다.
MILVUS_DB_USER = "root" 
MILVUS_DB_PWD = "Milvus"
MILVUS_DB_PORT = "19530"


컬렉션 명을 지정합니다.   
규칙 : kr_covid19_랩번호

In [ ]:
COLLECTION_NAME="kr_genai_lab_<YOUR NAME>"
PDF_FILE_NAME='KR-TMT2024-GEN-AI.pdf'

## Using Langchain

In [ ]:
# Import library
from ibm_watson_studio_lib import access_project_or_space
from langchain.chains import RetrievalQA

IBM Cloud의 watsonx 서비스에서 프로젝트 토큰 정보를 가져옵니다.   
메뉴는 `Project > Manage > Access Contorl > Access Tokens`입니다. 생성된 토큰 정보를 복사하여 `<YOUR ACCESS TOKEN HERE>`값을 삭제하고 여기에 붙여넣습니다.   

In [ ]:
# Create access token in project
token = "<YOUR ACCESS TOKEN HERE>"
wslib = access_project_or_space({"token":token})
wslib.download_file(PDF_FILE_NAME)

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter


loader = PyPDFLoader(PDF_FILE_NAME)
pages = loader.load_and_split()

In [ ]:
len(pages)

청크는 기본적으로 문자수를 가지고 길이를 측정합니다. chunk_size는 분할될 최대 문자수입니다.
* 텍스트 분할 방법: 단일 문자 기준.
* 청크 크기 측정 방법: 문자 수 기준.

In [ ]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=10000, 
    chunk_overlap=0)
docs = text_splitter.split_documents(pages)

In [ ]:
len(docs)


청크 내용을 확인합니다.

In [ ]:
i=0
for doc in docs:
    i+=1
    print(">청크 번호 : ",i)
    print(">청크 사이즈 : ",len(docs[i-1].page_content))
    print(">청크 내용 : ")
    print(doc.page_content)
    print("\n")

임베딩 모델을 지정합니다. 한글 지원이 되는 ```distiluse-base-multilingual-cased-v1```를 지정합니다.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# embedding_model_name = "distiluse-base-multilingual-cased-v1"
embedding_model_name = "intfloat/multilingual-e5-base"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

In [ ]:
len(docs)

In [ ]:
print(docs)

랭체인 Milvus를 객체를 이용하여 문서를 벡터 디비에 인서트 합니다.  

In [ ]:
vector_db_milvus = Milvus.from_documents(
    docs,
    embeddings,
    connection_args={"host": MILVUS_DB_HOST, 
                     "port": MILVUS_DB_PORT,
                     "user": MILVUS_DB_USER,
                     "password": MILVUS_DB_PWD
                    },
    collection_name = COLLECTION_NAME, ## custom collection name 
    search_params = {"metric":"IP","offset":0}, ## search params
)

옵션) 저장된 정보를 조회하기 위해 사용

In [ ]:
# vector_db_milvus = Milvus(
#             embeddings,
#             connection_args={"host": MILVUS_DB_HOST, "port": MILVUS_DB_PORT},
#             collection_name = COLLECTION_NAME, ## custom collection name 
#             search_params = {"metric":"IP","offset":0}, ## search params
# )

Milvus DB에 저장된 데이터를 조회 합니다.

In [ ]:
query = "기업의 생성형AI 도입 현황은?"
docs = vector_db_milvus.similarity_search(
    query,
    k=2
    )

In [ ]:
len(docs)

In [ ]:
for doc in docs:
    print(doc)
    print()

# MilvusDB를 적용한 RAG 시스템 구현
MilvusDB는 벡터 검색 및 분석에 특화된 NoSQL 데이터베이스입니다. 텍스트, 이미지, 오디오 등 다양한 데이터를 벡터 형태로 저장하고 효율적인 검색을 제공합니다.   
1. 문서 벡터화: MilvusDB에 저장할 문서를 벡터 형태로 변환합니다. 텍스트 문서의 경우 TF-IDF, Word2Vec, BERT 등 다양한 방법을 사용하여 벡터화할 수 있습니다.
2. MilvusDB에 데이터 저장: 벡터화된 문서를 MilvusDB에 저장합니다. MilvusDB는 벡터 검색 속도가 빠르기 때문에 질의와 관련된 문서를 빠르게 찾을 수 있습니다.
3. 질의 벡터화: 사용자 질의를 벡터 형태로 변환합니다. 질의 벡터화 방법은 문서 벡터화 방법과 동일할 수 있습니다.
4. MilvusDB에서 유사 문서 검색: 질의 벡터를 기반으로 MilvusDB에서 유사한 문서를 검색합니다. 검색된 문서는 RAG 시스템에서 새로운 텍스트를 생성하는 데 사용됩니다.
새로운 텍스트 생성: 검색된 문서를 기반으로 새로운 텍스트를 생성합니다. Transformer 기반 모델, Seq2Seq 모델 등 다양한 모델을 사용하여 새로운 텍스트를 생성할 수 있습니다.

In [ ]:
from pymilvus import (
    connections,
    Collection
)
from sentence_transformers import SentenceTransformer
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM




In [ ]:
def milvus_search(query, COLLECTION_NAME,embedding_model_name):
    model_name = embedding_model_name
    collection_name = COLLECTION_NAME
    connections.connect("default", host=MILVUS_DB_HOST, port=MILVUS_DB_PORT, user=MILVUS_DB_USER, password=MILVUS_DB_PWD )  
    collection = Collection(collection_name)
    collection.load()
    search_params = {
        "metric_type": "L2",
        "params": {"ef": 10},
    }
    model = SentenceTransformer(model_name)
    vectors_to_search = model.encode([query]).tolist()

    result = collection.search(vectors_to_search, "vector", search_params,
                                limit=3,
                                output_fields=["text", "vector"],
                                )

    hits = result[0]
    def text2doc(t):
        return Document(page_content = t)

    docs = [text2doc(h.entity.get('text')) for h in hits]
    return docs

In [ ]:
api_key = "<CLOUD_API_KEY>"
# region에 따라 주소가 다를 수 있습니다. 주소를 확인해 주세요.
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com" 
project_id = "<MY_PROJECT_ID>"

if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("One or more environment variables are missing!")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [ ]:
# watsonx model 초기화
params = {
#     GenParams.DECODING_METHOD: "sample",
#     GenParams.TEMPERATURE: 0.2,
#     GenParams.TOP_P: 1,
#     GenParams.TOP_K: 100,
    GenParams.DECODING_METHOD: "greedy",
    GenParams.TEMPERATURE:  0.7,
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.MAX_NEW_TOKENS: 1000
}

model_llm = Model(
    model_id="meta-llama/llama-3-3-70b-instruct",
    params=params,
    credentials=creds,
    project_id=project_id
).to_langchain()

In [ ]:
user_input = '기업의 생성형AI 도입 현황은?'
# user_input = "기업에서 생성형 AI를 도입하는 가치는?"
# user_input = "기업에서 생성형 AI를 도입하기 위해 고려사항은?"
# user_input = "기업에서 생성형 AI 도입을 가로막는 가장 큰 장애물은?"

docs_search = milvus_search(user_input,COLLECTION_NAME,embedding_model_name)
print(docs_search)

In [ ]:
chain_types = "stuff"
# chain_types = "map_reduce"
# chain_types = "refine"

chain = load_qa_chain(model_llm, chain_type=chain_types)
response = chain.run(input_documents=docs_search, question=user_input+". 한국어로 답하시오.")
print(response)

# 자원삭제

핸즈온을 완료후 자신이 생성한 콜렉션을 삭제합니다.

In [ ]:
from pymilvus import (
    connections,
    Collection
)
connections.connect(
  alias="default",
  host=MILVUS_DB_HOST, # YOUR IP
  port=MILVUS_DB_PORT,      # YOUR PORT
  user=MILVUS_DB_USER,
  password=MILVUS_DB_PWD
)


utility.drop_collection(COLLECTION_NAME)

connections.disconnect("default")